<a href="https://colab.research.google.com/github/Ksenia-90/recommender_systems/blob/lesson_2/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 4.3 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3413728 sha256=08c6764eb281512bbb23834ee954a7baeed14f5530572d1706b137b40db02021
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [8]:
data  = pd.read_csv("/content/retail_train.csv")
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [ ]:
#data = pd.read_csv('data/retail_train.csv')
#data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [9]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    recs = np.random.choice(items_weights.item_id,
                            p=items_weights.weight, 
                            size=n,
                            replace=False)
    return recs.tolist()

In [14]:
def get_weight_matrix(data):

    items_weight = data.groupby('item_id')['sales_value'].sum().reset_index()
    log_value = items_weight.sales_value.apply(lambda x: 0 if x < 1 else np.log(x))
    log_value_sum = log_value.sum()
    items_weight['weight'] = log_value.apply(lambda x: x / log_value_sum)

    return items_weight

In [15]:
%%time
items_weight = get_weight_matrix(data)
items_weight.head() 
# your_code

CPU times: user 58 ms, sys: 1.53 ms, total: 59.5 ms
Wall time: 67.7 ms


In [16]:
#проверка
items_weight.weight.sum()

1.0

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [ ]:
result_random_recommendation = []
result_popular_recommendation = []
result_weighted_random_recs = []
 # случайные рекомендации
    random_recommendations = [random_recommendation(top_items_ids, n=5) for user_id in user_ids]
    precisions_mean = average_precision_at_k(random_recommendations, actual, k=5)
    result_random_recommendation.append(precisions_mean)
    
    # рекомендация топ-5 товаров всем пользователям
    popular_recomendations = [popular_recs for user_id in user_ids]
    precisions_mean = average_precision_at_k(popular_recomendations, actual, k=5)
    result_popular_recommendation.append(precisions_mean)

    # взвешенные случайные рекомендации
    items_weights = get_weight_matrix(data.loc[data['item_id'].isin(top_items_ids), :])
    weighted_random_recommendations = [weighted_random_recommendation(items_weights, n=5) for user_id in user_ids]
    precisions_mean = average_precision_at_k(weighted_random_recommendations, actual, k=5)
    result_weighted_random_recs.append(precisions_mean)